In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [2]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
model_name = "csebuetnlp/mT5_m2m_crossSum"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


# New section

In [5]:
import pandas as pd

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
import zipfile

zip_file_path = '/content/sample_data/text_summarization.csv.zip'
extraction_dir = '/content/sample_data/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

print("Zip file extracted successfully.")


Zip file extracted successfully.


In [8]:
d = pd.read_csv('/content/sample_data/text_summarization.csv')
d.head()


,category,summary,text
0,technology,অ্যাপসে মিলবে ঢাকাসহ তিন জেলা আদালতের তথ্য,"ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙ..."
1,bangladesh,বিজ্ঞান ও প্রকৌশলে মার্কিন সর্বোচ্চ সম্মাননা...,যুক্তরাষ্ট্রে বিজ্ঞান ও প্রকৌশলে পেশা শুরুর প...
2,bangladesh,বিকল্প শিশুখাদ্য গ্রহণে শিশুর মৃত্যু হলে শাস্ত...,বিকল্প শিশুখাদ্য ও বাণিজ্যিকভাবে উত্পাদিত শিশু...
3,bangladesh,ট্রেনে কাটা পড়ে সাবেক সিভিল সার্জনের মৃত্যু,বগুড়ার আদমদীঘির সান্তাহারে গতকাল শনিবার ট্রেন...
4,bangladesh,যাত্রাবাড়ীতে চুলা জ্বালাতে গিয়ে দুই কর্মচারী ...,যাত্রাবাড়ীর একটি রেস্তোরাঁয় গতকাল বুধবার ভোর...


# New Section

In [9]:
 data = d.drop(columns=['category'])
 data = data.head(10000)
 data

,summary,text
0,অ্যাপসে মিলবে ঢাকাসহ তিন জেলা আদালতের তথ্য,"ঢাকা মহানগর ও ঢাকা জেলা আদালত, কিশোরগঞ্জ ও রাঙ..."
1,বিজ্ঞান ও প্রকৌশলে মার্কিন সর্বোচ্চ সম্মাননা...,যুক্তরাষ্ট্রে বিজ্ঞান ও প্রকৌশলে পেশা শুরুর প...
2,বিকল্প শিশুখাদ্য গ্রহণে শিশুর মৃত্যু হলে শাস্ত...,বিকল্প শিশুখাদ্য ও বাণিজ্যিকভাবে উত্পাদিত শিশু...
3,ট্রেনে কাটা পড়ে সাবেক সিভিল সার্জনের মৃত্যু,বগুড়ার আদমদীঘির সান্তাহারে গতকাল শনিবার ট্রেন...
4,যাত্রাবাড়ীতে চুলা জ্বালাতে গিয়ে দুই কর্মচারী ...,যাত্রাবাড়ীর একটি রেস্তোরাঁয় গতকাল বুধবার ভোর...
...,...,...
9995,স ং ক্ষি প্ত বি শ্ব স ং বা দ,বিধায়ক বহিষ্কারদলের সিদ্ধান্ত অমান্য করে গত শু...
9996,আমরা আরও অনেক দূর যেতে চাই: প্রধানমন্ত্রী,"প্রধানমন্ত্রী শেখ হাসিনা বলেছেন, ‘আমরা আরও অনে..."
9997,হাওরে ক্ষতিগ্রস্তদের মধ্যে মৃধা ফাউন্ডেশনের ত্...,কিশোরগঞ্জের মিঠামইনের কাটখাল ইউনিয়নে হাওরে ক্ষ...
9998,টেকনাফে ১ লাখ ৭০ হাজার ইয়াবা বড়ি জব্দ,কক্সবাজারের টেকনাফে বর্ডার গার্ড বাংলাদেশের (ব...


In [10]:
import re
import csv
import pandas as pd

# Load stop words from a CSV file
stop_words_file = '/content/sample_data/stopwords_bn.txt'
stop_word_list = []

with open(stop_words_file, 'r', encoding='utf-8') as temp_output_file:
    reader = csv.reader(temp_output_file, delimiter='\n')
    for row in reader:
        stop_word_list.append(row[0])

def preprocess(text):
    # Remove URLs

    non_bengali_pattern = r'[^\u0980-\u09FF\s]'
    text = re.sub(non_bengali_pattern, '', text)


    # Tokenize the text into words
    words = text.split()  # Splitting into words based on spaces

    # Remove stop words
    words = [word for word in words if word not in stop_word_list]


    preprocessed_text = ' '.join(words)
    return preprocessed_text

# Columns to preprocess
text_columns = ['summary', 'text']

# Apply preprocessing to text columns
for column in text_columns:
    data[column] = data[column].apply(preprocess)

In [11]:
X = list(data['text'])

In [12]:
y = list(data['summary'])


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Tokenize input and target sequences
input_encodings = tokenizer(X_train, max_length=1024, truncation=True, padding=True,return_tensors="pt")
target_encodings = tokenizer(y_train, max_length=70, truncation=True, padding=True,return_tensors="pt")
import torch
# Create PyTorch datasets
train_dataset = torch.utils.data.TensorDataset(input_encodings['input_ids'], input_encodings['attention_mask'],
                                              target_encodings['input_ids'], target_encodings['attention_mask'])


In [15]:
# Tokenize input and target sequences
input_encodings = tokenizer(X_test, max_length=1024, truncation=True, padding=True,return_tensors="pt")
target_encodings = tokenizer(y_test, max_length=70, truncation=True, padding=True,return_tensors="pt")
import torch
# Create PyTorch datasets
test_dataset = torch.utils.data.TensorDataset(input_encodings['input_ids'], input_encodings['attention_mask'],
                                              target_encodings['input_ids'], target_encodings['attention_mask'])

In [16]:


# Custom data collation function
def collate_fn(batch):
    input_ids, input_attention_mask, target_ids, target_attention_mask = zip(*batch)
    return {
        'input_ids': torch.stack(input_ids),
        'attention_mask': torch.stack(input_attention_mask),
        'labels': torch.stack(target_ids),
    }


# from transformers import DataCollatorForSeq2Seq

# Create a DataCollatorForSeq2Seq instance
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)





In [17]:
!pip uninstall  accelerate -y
!pip install  accelerate -U
from transformers import TrainingArguments, Trainer

Found existing installation: accelerate 0.22.0
Uninstalling accelerate-0.22.0:
  Successfully uninstalled accelerate-0.22.0
  Using cached accelerate-0.22.0-py3-none-any.whl (251 kB)


In [18]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=7,
    per_device_train_batch_size=10,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=2,
)

In [19]:
import torch
# Check if CUDA (GPU support) is available
torch.cuda.empty_cache()
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")
model.to(device)

CUDA is available. Using GPU.


MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [20]:

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset = test_dataset,
    data_collator=collate_fn
)

In [21]:
# Train the model
trainer.train()

Step,Training Loss
500,0.683700
1000,0.576400
1500,0.532600
2000,0.502000
2500,0.483900


TrainOutput(global_step=2800, training_loss=0.5471180997576033, metrics={'train_runtime': 10016.5981, 'train_samples_per_second': 5.591, 'train_steps_per_second': 0.28, 'total_flos': 1.34293166751744e+17, 'train_loss': 0.5471180997576033, 'epoch': 7.0})

In [22]:
trainer.evaluate()

{'eval_loss': 0.600790798664093,
 'eval_runtime': 54.7692,
 'eval_samples_per_second': 36.517,
 'eval_steps_per_second': 4.565,
 'epoch': 7.0}

In [27]:
model.save_pretrained("bangla-text-summarization-finetuned")

In [29]:
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json')